In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
from datetime import datetime
import sys
sys.path.append('../scripts')
from feature_transform import dates_encoder, merge_path, interpolate_missing_values, scrap_weather, merge_weather_data, get_distance

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
import haversine as hs
from haversine import Unit
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
%matplotlib inline

In [23]:
train = pd.read_csv('../data/flights_train.csv', parse_dates = ['flight_date']).reset_index(drop=True)
X_test = pd.read_csv('../data/flights_Xtest.csv', parse_dates = ['flight_date']).reset_index(drop=True)

In [37]:
#train = get_distance(train)
#X_test = get_distance(X_test)

In [24]:
train ['flight_date'].min(), train ['flight_date'].max(), X_test['flight_date'].min(), X_test['flight_date'].max()

(Timestamp('2011-09-01 00:00:00'),
 Timestamp('2012-11-14 00:00:00'),
 Timestamp('2012-11-15 00:00:00'),
 Timestamp('2013-03-05 00:00:00'))

In [25]:
train_weather = scrap_weather(train, start=datetime(2011, 9, 1), end=datetime(2012, 11, 14))
train_weather

,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,airport_code
0,2011-09-01,27.8,21.7,33.9,0.0,0.0,NaN,4.0,ATL
1,2011-09-02,27.1,19.4,35.0,0.0,0.0,NaN,3.5,ATL
2,2011-09-03,27.7,20.6,35.0,0.0,0.0,NaN,5.6,ATL
3,2011-09-04,23.7,20.6,26.7,6.1,0.0,NaN,4.4,ATL
4,2011-09-05,23.7,22.2,25.6,16.0,0.0,147.9,10.8,ATL
...,...,...,...,...,...,...,...,...,...
436,2012-11-10,11.6,8.9,15.0,0.0,0.0,282.1,16.9,SFO
437,2012-11-11,10.6,6.1,15.6,0.0,0.0,NaN,3.6,SFO
438,2012-11-12,11.5,6.1,17.2,0.0,0.0,NaN,5.0,SFO
439,2012-11-13,13.5,8.3,18.3,0.0,0.0,NaN,4.3,SFO


In [26]:
test_weather = scrap_weather(X_test, start=datetime(2012, 11, 15), end=datetime(2013, 3, 5))
test_weather

,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,airport_code
0,2012-11-15,7.6,5.0,11.7,0.0,0.0,NaN,6.3,ATL
1,2012-11-16,7.8,1.0,16.1,0.0,0.0,NaN,0.9,ATL
2,2012-11-17,7.4,0.6,16.1,0.0,0.0,66.2,5.7,ATL
3,2012-11-18,10.9,5.0,17.8,0.0,0.0,63.7,7.4,ATL
4,2012-11-19,10.6,2.8,15.6,0.0,0.0,23.6,2.7,ATL
...,...,...,...,...,...,...,...,...,...
106,2013-03-01,11.8,7.8,21.7,0.0,0.0,NaN,7.6,SFO
107,2013-03-02,12.8,10.6,17.2,0.0,0.0,NaN,9.4,SFO
108,2013-03-03,11.4,9.4,15.0,0.0,0.0,NaN,20.2,SFO
109,2013-03-04,9.6,7.8,13.3,0.0,0.0,NaN,15.5,SFO


In [27]:
train_weather = train_weather.drop(labels=['wdir'], axis=1)
test_weather = test_weather.drop(labels=['wdir'], axis=1)

In [28]:
train = merge_weather_data(X = train, weather_data = train_weather)
train

,flight_date,from,to,avg_weeks,target,std_weeks,tavg_from,prcp_from,wspd_from,snow_from,tavg_to,prcp_to,wspd_to,snow_to
0,2012-06-19,ORD,DFW,12.875000,12.331296,9.812647,30.0,0.0,24.8,0.0,27.5,0.0,29.2,0.0
1,2012-06-19,ORD,PHX,11.772727,10.502073,8.970490,30.0,0.0,24.8,0.0,35.4,0.0,12.2,NaN
2,2012-06-19,ORD,SFO,13.480000,12.160800,9.372477,30.0,0.0,24.8,0.0,15.7,0.0,16.9,0.0
3,2012-06-19,LAS,SFO,13.925926,10.377696,11.272587,33.8,0.0,17.6,0.0,15.7,0.0,16.9,0.0
4,2012-06-19,ORD,ATL,11.173913,11.788080,8.819669,30.0,0.0,24.8,0.0,24.6,0.0,3.6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8891,2012-09-16,JFK,LAS,18.973684,11.018431,11.679242,19.0,0.0,13.0,0.0,29.8,0.0,13.7,0.0
8892,2011-09-08,LAS,JFK,12.571429,10.649808,8.230155,32.0,0.0,9.4,0.0,21.8,9.7,15.5,0.0
8893,2011-09-15,BOS,PHL,7.133333,11.923892,4.688385,21.2,0.8,16.9,0.0,19.7,4.6,16.2,0.0
8894,2012-08-28,ORD,LGA,13.826087,13.316772,11.491705,23.1,0.0,8.3,0.0,26.9,1.5,19.1,0.0


In [29]:
X_test = merge_weather_data(X = X_test, weather_data = test_weather)
X_test

,flight_date,from,to,avg_weeks,std_weeks,tavg_from,prcp_from,wspd_from,snow_from,tavg_to,prcp_to,wspd_to,snow_to
0,2013-01-22,ATL,MCO,10.363636,8.232025,2.3,0.0,13.3,0.0,15.1,0.0,11.3,0.0
1,2013-02-22,ATL,DEN,8.294118,5.542616,6.6,37.6,14.8,0.0,-6.8,0.0,16.6,76.0
2,2013-02-22,SEA,DEN,7.400000,4.910630,6.1,9.4,29.2,0.0,-6.8,0.0,16.6,76.0
3,2013-02-22,MSP,DEN,10.200000,6.902326,-4.5,6.1,10.8,178.0,-6.8,0.0,16.6,76.0
4,2013-02-22,DFW,DEN,9.166667,6.400827,4.1,0.0,17.6,0.0,-6.8,0.0,16.6,76.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2227,2012-11-17,PHL,MCO,20.850000,13.463245,6.6,0.0,11.9,0.0,19.5,0.0,15.8,0.0
2228,2013-01-21,LAS,JFK,12.692308,7.928527,10.0,0.0,10.1,0.0,-1.3,0.8,13.7,0.0
2229,2012-12-01,PHX,DEN,9.500000,6.997899,17.9,0.0,7.2,NaN,9.5,0.0,15.5,0.0
2230,2012-12-08,MIA,JFK,13.000000,10.291406,24.4,0.5,12.2,NaN,8.9,8.6,9.4,0.0


In [30]:
train = train.sort_values(by='flight_date', ascending=True)
X_test = X_test.sort_values(by='flight_date', ascending=True)

In [31]:
train.isnull().sum()

flight_date       0
from              0
to                0
avg_weeks         0
target            0
std_weeks         0
tavg_from        10
prcp_from         0
wspd_from         0
snow_from      1162
tavg_to          11
prcp_to           0
wspd_to           0
snow_to        1201
dtype: int64

In [32]:
X_test.isnull().sum()

flight_date      0
from             0
to               0
avg_weeks        0
std_weeks        0
tavg_from       10
prcp_from        0
wspd_from        0
snow_from      298
tavg_to         10
prcp_to          0
wspd_to          0
snow_to        284
dtype: int64

In [33]:
train['snow_to'] = train['snow_to'].fillna(0)
train['snow_from'] = train['snow_from'].fillna(0)
X_test['snow_to'] = X_test['snow_to'].fillna(0)
X_test['snow_from'] = X_test['snow_from'].fillna(0)

In [34]:
train = interpolate_missing_values(train, column='tavg_from', rename = 'tavg_from_filled')
train = interpolate_missing_values(train, column='tavg_to', rename = 'tavg_to_filled')

In [35]:
X_test = interpolate_missing_values(X_test, column='tavg_from', rename = 'tavg_from_filled')
X_test = interpolate_missing_values(X_test, column='tavg_to', rename = 'tavg_to_filled')

In [36]:
train.isnull().sum()

flight_date    0
from           0
to             0
avg_weeks      0
target         0
std_weeks      0
prcp_from      0
wspd_from      0
snow_from      0
prcp_to        0
wspd_to        0
snow_to        0
tavg_from      0
tavg_to        0
dtype: int64

# Process data for pycaret evaluation

In [29]:
train_processed = dates_encoder(train)
X_test_processed = dates_encoder(X_test)

In [18]:
train_processed.to_csv('../data/flights_train_weather_distance_processed.csv')
X_test_processed.to_csv('../data/flights_X_test_weather_distance_processed.csv')

# Model

In [37]:
train = merge_path(train)
X_test = merge_path(X_test)

date_encoder = FunctionTransformer(dates_encoder)
date_cols = ["flight_date"]


categorical_encoder = OneHotEncoder(handle_unknown="ignore", sparse = False)
categorical_cols = ["path"]

preprocessor = make_column_transformer(
        (date_encoder, date_cols),
        (categorical_encoder, categorical_cols),
        remainder='passthrough'
    )

In [38]:
regressor_ = LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=8945, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

regressor = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=-1, oob_score=False,
                       random_state=8945, verbose=0, warm_start=False)
regressor_cb = CatBoostRegressor()
regressor_xgb = XGBRegressor()

In [39]:
pipe_line = make_pipeline(preprocessor, regressor)
pipe_line_ = make_pipeline(preprocessor, regressor_)
pipe_line_xgb = make_pipeline(preprocessor, regressor_xgb)
pipe_line_cb = make_pipeline(preprocessor, regressor_cb)

In [ ]:
X = train.drop('target', axis = 1)
y = train.target

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=30)

pipe_line.fit(train_X,train_y)
pipe_line_.fit(train_X,train_y)
pipe_line_xgb.fit(train_X,train_y)
pipe_line_cb.fit(train_X,train_y)

In [44]:
y_pred = pipe_line_cb.predict(test_X)

print(f'The RMSE score is: {np.sqrt(mean_squared_error(test_y, y_pred))}')

The RMSE score is: 0.3747697383119491


In [58]:
X_test

,flight_date,avg_weeks,std_weeks,distance,prcp_from,wspd_from,prcp_to,wspd_to,tavg_from,tavg_to
2097,2012-11-15,12.000000,7.956403,1925.940575,0.0,10.8,0.0,6.3,1.9,7.6
783,2012-11-15,9.900000,6.608607,1290.781744,0.0,7.6,0.0,8.6,10.7,2.8
781,2012-11-15,9.736842,8.312254,1536.909538,0.0,8.6,0.0,6.1,5.8,16.1
780,2012-11-15,10.909091,8.922465,1983.272852,0.0,7.6,0.0,6.1,10.7,16.1
2100,2012-11-15,10.150000,8.536454,1459.056790,0.0,12.6,0.0,6.3,5.6,7.6
...,...,...,...,...,...,...,...,...,...,...
459,2013-03-05,11.100000,8.589467,4193.040474,0.0,12.2,0.0,11.1,13.5,2.9
460,2013-03-05,10.111111,10.493073,664.841940,3.6,22.0,0.0,11.2,10.2,16.4
1821,2013-03-05,4.636364,4.249064,1088.308056,0.0,10.8,16.8,19.1,3.7,-0.3
1991,2013-03-05,5.916667,4.316108,2231.344813,0.0,12.6,0.0,32.8,4.3,10.5


In [57]:
submission = pipe_line.predict(X_test)
submission = pd.DataFrame(submission)
submission.to_csv('../submissions/FPX_submission_random_forest_weather_distance.csv', index = False, header = False)